<a href="https://colab.research.google.com/github/mmtog/Regresi-n-Log-stica/blob/main/regresi%C3%B3n_log%C3%ADstica_Grecia_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Cargamos librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
grecia =pd.read_csv("Grecia_limpios_final.csv")
grecia.head(5)

,Unnamed: 0,host_total_listings_count,accommodates,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,...,host_identity_verified,neighbourhood_cleansed,property_type,room_type,bathrooms,bedrooms,amenities,calendar_last_scraped,id,host_id
0,0,10.0,3.5,108.0,1.0,180.0,1.0,4.0,180.0,180.0,...,t,ΑΜΠΕΛΟΚΗΠΟΙ,Entire condo,Entire home/apt,2.0,3.0,"[""Ethernet connection"", ""Room-darkening shades...",2024-06-26,10595,37177
1,1,10.0,4.0,136.0,1.0,1125.0,1.0,4.0,1125.0,1125.0,...,t,ΑΜΠΕΛΟΚΗΠΟΙ,Entire rental unit,Entire home/apt,1.0,1.0,"[""Ethernet connection"", ""43 inch HDTV"", ""Priva...",2024-06-26,10990,37177
2,2,10.0,2.0,67.0,1.0,90.0,1.0,3.0,90.0,90.0,...,t,ΑΜΠΕΛΟΚΗΠΟΙ,Entire rental unit,Entire home/apt,1.0,0.0,"[""Ethernet connection"", ""43 inch HDTV"", ""Priva...",2024-06-26,10993,37177
3,3,10.0,4.0,78.0,1.0,45.0,1.0,3.0,45.0,45.0,...,t,ΑΜΠΕΛΟΚΗΠΟΙ,Entire rental unit,Entire home/apt,1.0,1.0,"[""Ethernet connection"", ""Private patio or balc...",2024-06-26,10995,37177
4,4,10.0,2.0,58.0,1.0,1125.0,1.0,4.0,1125.0,1125.0,...,t,ΑΜΠΕΛΟΚΗΠΟΙ,Entire rental unit,Entire home/apt,1.0,1.0,"[""Other electric stove"", ""Room-darkening shade...",2024-06-26,27262,37177


In [3]:
# Reemplazar 'unknown' por NaN
grecia['bathrooms'] = grecia['bathrooms'].replace('Unknown', np.nan)
grecia['bedrooms'] = grecia['bedrooms'].replace('Unknown', np.nan)
# convertir a float
grecia['bathrooms'] = grecia['bathrooms'].astype(float)
grecia['bedrooms'] = grecia['bedrooms'].astype(float)

In [4]:
#Rellenamos valores nulos
grecia= grecia.fillna(method='bfill')
greciaf= grecia.fillna(method='ffill')

<ipython-input-4-fd68f459e81f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grecia= grecia.fillna(method='bfill')
<ipython-input-4-fd68f459e81f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  greciaf= grecia.fillna(method='ffill')


In [5]:
grecia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13274 entries, 0 to 13273
Data columns (total 44 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 13274 non-null  int64  
 1   host_total_listings_count  13274 non-null  float64
 2   accommodates               13274 non-null  float64
 3   price                      13274 non-null  float64
 4   minimum_nights             13274 non-null  float64
 5   maximum_nights             13274 non-null  float64
 6   minimum_minimum_nights     13274 non-null  float64
 7   maximum_minimum_nights     13274 non-null  float64
 8   minimum_maximum_nights     13274 non-null  float64
 9   maximum_maximum_nights     13274 non-null  float64
 10  minimum_nights_avg_ntm     13274 non-null  float64
 11  maximum_nights_avg_ntm     13274 non-null  float64
 12  availability_30            13274 non-null  int64  
 13  availability_60            13274 non-null  int

In [6]:
grecia['number_of_reviews'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 13274 entries, 0 to 13273
Series name: number_of_reviews
Non-Null Count  Dtype  
--------------  -----  
13274 non-null  float64
dtypes: float64(1)
memory usage: 103.8 KB


#** Regre Log 1 **



In [7]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=grecia['number_of_reviews'].max()
Min=grecia['number_of_reviews'].min()
Limites= [Min, Max]
Limites

[0.0, 140.0]

In [8]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos=np.linspace(-0.9, 140.1, 3)
intervalos

array([ -0.9,  69.6, 140.1])

In [9]:
#Creamos las categorías
categorias= ["pocas", "muchas"]

In [10]:
#Finalmente creamos las categorías en la columna numérica
grecia['number_of_reviews']=pd.cut(x= grecia['number_of_reviews'], bins=intervalos, labels= categorias )

In [11]:
#Rellenamos valores nulos
grecia= grecia.fillna(method='bfill')
greciaf= grecia.fillna(method='ffill')

<ipython-input-11-fd68f459e81f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grecia= grecia.fillna(method='bfill')
<ipython-input-11-fd68f459e81f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  greciaf= grecia.fillna(method='ffill')


In [12]:
grecia['number_of_reviews'].isnull().sum()

0

In [13]:
nulos= grecia.isnull().sum().sum()
nulos

0

In [14]:
vars_Indep = grecia[['bedrooms']]
vars_Dep = grecia ['number_of_reviews']

In [15]:
#Redefinimos las variables
x= vars_Indep
y= vars_Dep

In [16]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test, X_train, Y_test, Y_train= train_test_split(x,y, test_size=0.3, random_state=None)

In [17]:
escalar = StandardScaler()

In [18]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [19]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [20]:
#Entrenamos el modelo
algoritmo.fit(X_train, Y_train)

LogisticRegression()

In [21]:
#realizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array(['pocas', 'pocas', 'pocas', ..., 'pocas', 'pocas', 'pocas'],
      dtype=object)

In [22]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test, y_pred)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[   0  945]
 [   1 8345]]


In [23]:
#calculamos la prediccion del modelo
from sklearn.metrics import precision_score

precision = precision_score(Y_test, y_pred, average="binary", pos_label= "pocas")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.8982777179763186


In [24]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(Y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8981810354106124


In [26]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(Y_test, y_pred, average="binary", pos_label="pocas")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9998801821231728


# **2**

In [26]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=grecia['review_scores_cleanliness'].max()
Min=grecia['review_scores_cleanliness'].min()
Limites= [Min, Max]
Limites

[3.78, 5.0]

In [27]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos1=np.linspace(3.77, 5.1,3)
intervalos1

array([3.77 , 4.435, 5.1  ])

In [28]:
#Creamos las categorías
categorias1= ["sucio", "limpio"]

In [29]:
grecia['review_scores_cleanliness']

,review_scores_cleanliness
0,4.85
1,4.86
2,4.94
3,4.90
4,4.96
...,...
13269,4.80
13270,4.80
13271,4.80
13272,4.80


In [30]:
#Finalmente creamos las categorías en la columna numérica
grecia['review_scores_cleanliness']=pd.cut(x= grecia['review_scores_cleanliness'], bins=intervalos1, labels= categorias1 )

In [31]:
#Rellenamos valores nulos
grecia= grecia.fillna(method='bfill')
grecia= grecia.fillna(method='ffill')

<ipython-input-31-8660068a4c2f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grecia= grecia.fillna(method='bfill')
<ipython-input-31-8660068a4c2f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grecia= grecia.fillna(method='ffill')


In [32]:
grecia['review_scores_cleanliness'].isnull().sum()

0

In [33]:
vars_Indep_1 = grecia[['bedrooms']]
vars_Dep_1 = grecia['review_scores_cleanliness']

In [34]:
#Redefinimos las variables
x= vars_Indep_1
y= vars_Dep_1

In [35]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test1, X_train1, Y_test1, Y_train1= train_test_split(x,y, test_size=0.3, random_state=None)

In [36]:
escalar1 = StandardScaler()

In [37]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train1 = escalar.fit_transform(X_train1)
X_test1 = escalar.transform(X_test1)

In [38]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo1 = LogisticRegression()

In [39]:
#Entrenamos el modelo
algoritmo1.fit(X_train1, Y_train1)

LogisticRegression()

In [40]:
#realizamos una prediccion
y_pred1 = algoritmo1.predict(X_test1)
y_pred1

array(['limpio', 'limpio', 'limpio', ..., 'limpio', 'limpio', 'limpio'],
      dtype=object)

In [41]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test1, y_pred1)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[8677    0]
 [ 614    0]]


In [42]:
#Calculamos la prediccion del modelo
from sklearn.metrics import precision_score

precision = precision_score(Y_test1, y_pred1, average="binary", pos_label="limpio")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.933914540953611


In [43]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(Y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.933914540953611


In [44]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(Y_test1, y_pred1, average="binary", pos_label="limpio")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


## **3**

In [45]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=grecia['accommodates'].max()
Min=grecia['accommodates'].min()
Limites= [Min, Max]
Limites

[1.0, 7.0]

In [46]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos2=np.linspace(.99,7.1,3)
intervalos2

array([0.99 , 4.045, 7.1  ])

In [47]:
#Creamos las categorías
categorias2= ["incomodo", "comodo"]

In [48]:
grecia['accommodates']

,accommodates
0,3.5
1,4.0
2,2.0
3,4.0
4,2.0
...,...
13269,4.0
13270,2.0
13271,2.0
13272,2.0


In [49]:
#Finalmente creamos las categorías en la columna numérica
grecia['accommodates']=pd.cut(x= grecia['accommodates'], bins=intervalos2, labels= categorias2 )

In [50]:
#Rellenamos valores nulos
grecia= grecia.fillna(method='bfill')
greciaf= grecia.fillna(method='ffill')

<ipython-input-50-fd68f459e81f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grecia= grecia.fillna(method='bfill')
<ipython-input-50-fd68f459e81f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  greciaf= grecia.fillna(method='ffill')


In [51]:
grecia['accommodates'].isnull().sum()

0

In [52]:
vars_Indep_2 = grecia[['bedrooms']]

vars_Dep_2 = grecia['accommodates']

In [53]:
#Redefinimos las variables
x= vars_Indep_2
y= vars_Dep_2

In [54]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test2, X_train2, Y_test2, Y_train2= train_test_split(x,y, test_size=0.3, random_state=None)

In [55]:
escalar2 = StandardScaler()

In [56]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train2 = escalar.fit_transform(X_train2)
X_test2 = escalar.transform(X_test2)

In [57]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo2 = LogisticRegression()

In [58]:
#Entrenamos el modelo
algoritmo2.fit(X_train2, Y_train2)

LogisticRegression()

In [59]:
#realizamos una prediccion
y_pred2 = algoritmo2.predict(X_test2)
y_pred2

array(['incomodo', 'incomodo', 'incomodo', ..., 'incomodo', 'incomodo',
       'incomodo'], dtype=object)

In [60]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test2, y_pred2)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[ 329 1331]
 [ 316 7315]]


In [61]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score
precision = precision_score(Y_test2, y_pred2, average="binary", pos_label="incomodo")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.8460559796437659


In [62]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test2, y_pred2)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8227316758153052


In [63]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test2, y_pred2, average="binary", pos_label="incomodo")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.958589961997117


## **4**

In [64]:
#Ajustar maximo de filas
pd.options.display.max_rows= None

In [65]:
grecia['bathrooms'] = pd.to_numeric(grecia['bathrooms'], errors='coerce')

In [66]:
grecia = grecia.dropna(subset=['price'])

In [67]:
grecia['price'] = pd.to_numeric(grecia['price'], errors='coerce')

# Eliminar los valores que quedaron como NaN después de la conversión
grecia = grecia.dropna(subset=['price'])


In [68]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max = grecia['price'].max()
Min = grecia['price'].min()
Limites = [Min, Max]
Limites

[11.0, 214.0]

In [69]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos3=np.linspace(10.9,214.1, 3)
intervalos3

array([ 10.9, 112.5, 214.1])

In [70]:
#Creamos las categorías
categorias3= ["barato", "caro"]

In [71]:
#Finalmente creamos las categorías en la columna numérica
grecia['price']=pd.cut(x= grecia['price'], bins=intervalos3, labels= categorias3 )

In [72]:
#Rellenamos valores nulos
grecia= grecia.fillna(method='bfill')
grecia= grecia.fillna(method='ffill')

<ipython-input-72-8660068a4c2f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grecia= grecia.fillna(method='bfill')
<ipython-input-72-8660068a4c2f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grecia= grecia.fillna(method='ffill')


In [73]:
grecia['price'].isnull().sum()

0

In [74]:
vars_Indep_3 = grecia[['bathrooms']]

vars_Dep_3 = grecia['price']

In [75]:
#Redefinimos las variables
x= vars_Indep_3
y= vars_Dep_3

In [76]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test3, X_train3, Y_test3, Y_train3= train_test_split(x,y, test_size=0.3, random_state=None)

In [77]:
escalar3 = StandardScaler()

In [78]:
X_train3 = escalar3.fit_transform(X_train3) # use escalar3 instead of escalar
X_test3 = escalar3.transform(X_test3) # use escalar3 instead of escalar

In [79]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo3 = LogisticRegression()

In [80]:
#Entrenamos el modelo
algoritmo3.fit(X_train3, Y_train3)

LogisticRegression()

In [81]:
#realizamos una prediccion
y_pred3 = algoritmo3.predict(X_test3)
y_pred3

array(['barato', 'barato', 'barato', ..., 'barato', 'barato', 'barato'],
      dtype=object)

In [82]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test3, y_pred3)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[7494    2]
 [1795    0]]


In [83]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score
precision = precision_score(Y_test3, y_pred3, average="binary", pos_label="barato")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.8067606846808053


In [84]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test3, y_pred3)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8065870196964805


In [85]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test3, y_pred3, average="binary", pos_label="barato")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9997331910352187


## **5**

In [27]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=grecia['review_scores_rating'].max()
Min=grecia['review_scores_rating'].min()
Limites= [Min, Max]
Limites

[3.82, 5.0]

In [28]:
grecia['review_scores_rating']

,review_scores_rating
0,4.86
1,4.82
2,4.83
3,4.81
4,4.84
...,...
13269,4.80
13270,4.80
13271,4.80
13272,4.80


In [29]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos4=np.linspace(3.81, 5.01, 3)
intervalos4

array([3.81, 4.41, 5.01])

In [30]:
#Creamos las categorías
categorias4= ["malo", "bueno"]

In [31]:
grecia['review_scores_rating'] = pd.to_numeric(grecia['review_scores_rating'], errors='coerce')

In [32]:
#Finalmente creamos las categorías en la columna numérica
grecia['review_scores_rating']=pd.cut(x= grecia['review_scores_rating'], bins=intervalos4, labels= categorias4 )

In [33]:
#Rellenamos valores nulos
grecia= grecia.fillna(method='bfill')
grecia= grecia.fillna(method='ffill')

<ipython-input-33-8660068a4c2f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grecia= grecia.fillna(method='bfill')
<ipython-input-33-8660068a4c2f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grecia= grecia.fillna(method='ffill')


In [34]:
grecia['review_scores_rating'].isnull().sum()

0

In [35]:
grecia['bedrooms'] = pd.to_numeric(grecia['bedrooms'], errors='coerce')

In [36]:
vars_Indep_4 = grecia[['bedrooms']]

vars_Dep_4 = grecia['review_scores_rating']

In [37]:
#Redefinimos las variables
x= vars_Indep_4
y= vars_Dep_4

In [38]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test4, X_train4, Y_test4, Y_train4= train_test_split(x,y, test_size=0.3, random_state=None)

In [39]:
escalar4 = StandardScaler()

In [40]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train4 = escalar.fit_transform(X_train4)
X_test4 = escalar.transform(X_test4)

In [41]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo4 = LogisticRegression()

In [42]:
#Entrenamos el modelo
algoritmo4.fit(X_train4, Y_train4)

LogisticRegression()

In [43]:
#Entrenamos el modelo
algoritmo4.fit(X_train4, Y_train4)

LogisticRegression()

In [44]:
#realizamos una prediccion
y_pred4 = algoritmo4.predict(X_test4)
y_pred4

array(['bueno', 'bueno', 'bueno', ..., 'bueno', 'bueno', 'bueno'],
      dtype=object)

In [45]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test4, y_pred4)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[8719    0]
 [ 572    0]]


In [46]:
#Calculamos la prediccion del modelo
from sklearn.metrics import precision_score
precision = precision_score(Y_test4, y_pred4, average="binary", pos_label="malo")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [47]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test4, y_pred4)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9384350446668819


In [48]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test4, y_pred4, average="binary", pos_label="malo")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0
